In [34]:
import pandas as pd
import openpyxl, pprint, os, sys
from datetime import datetime, timedelta
from openpyxl.styles import Alignment

In [35]:
# csvファイルを読み込む
products = pd.read_csv("../input/products.csv", encoding="shift_jis")
orders = pd.read_csv("../input/orders.csv", encoding="shift_jis")

In [36]:
# テンプレートを読み込む
quotation_wb = openpyxl.load_workbook("../template/quotation_template.xlsx")
# 対象のワークシートを読み込む
quotation_ws = quotation_wb["Quotation"]

In [37]:
# 対象の注文IDを対話形式で入力
target_id = input("注文IDを入力して下さい")

注文IDを入力して下さいO0003


In [38]:
# 情報を元に見積書を作成
# 対象の取引データを抽出
target_orders = orders[orders["order_id"] == target_id]
target_orders

order_id customer_name order_date product_id  quantity  \
6    O0003           鈴木健  2025/5/11       P006         3   
7    O0003           鈴木健  2025/5/11       P002         5   

  invoice_registration_number  
6              T1234567890003  
7              T1234567890003

In [39]:
# データを結合
merged = target_orders.merge(products, on="product_id", how="left")

In [40]:
# 金額列の小計
merged["subtotal"]  = merged["unit_price"] * merged["quantity"]

In [41]:
# 消費税
merged["VAT"] = merged["subtotal"] * merged["tax_rate"] / 100
merged["VAT"] = merged["VAT"].round(0).astype(int)

In [42]:
# 合計
merged["total_amount"] = merged["subtotal"] + merged["VAT"]
merged["total_amount"] = merged["total_amount"].astype(int)

In [43]:
# 見積書への入力作業
# 明細の開始行
start_row = 19

# 右詰めスタイルを作成
right_align = Alignment(horizontal="right")

# 日付設定
today = datetime.today()
due_date = today + timedelta(days=30)

# 明細行描写
for i, (_, row) in enumerate(merged.iterrows()):
    # No(連番)
    quotation_ws.cell(row=start_row + i, column=2, value=i + 1) 
    # 商品名
    quotation_ws.cell(row=start_row + i, column=3, value=row["product_name"]) 
     # 数量
    quotation_ws.cell(row=start_row + i, column=4, value=row["quantity"])
    # 単価(円)
    cell = quotation_ws.cell(row=start_row + i, column=5, value=row["unit_price"]) 
    cell.number_format = '"¥"#,##0'
    # 金額(円)
    cell = quotation_ws.cell(row=start_row + i, column=6, value=row["subtotal"]) 
    cell.number_format = '"¥"#,##0'
    
# 小計金額
cell = quotation_ws.cell(row=35, column=6, value=merged["subtotal"].sum()) 
cell.number_format = '"¥"#,##0'

# 消費税
cell = quotation_ws.cell(row=36, column=6, value=merged["VAT"].sum()) 
cell.number_format = '"¥"#,##0'

# 合計金額
cell = quotation_ws.cell(row=37, column=6, value=merged["total_amount"].sum()) 
cell.number_format = '"¥"#,##0'

# 宛名
quotation_ws["B3"] = merged.iloc[0]["customer_name"] + "  様"

# 見積日
cell = quotation_ws.cell(row=12, column=6, value=today.strftime("%Y年%m月%d日")) 
cell.alignment = right_align

# 振込期限
cell = quotation_ws.cell(row=13, column=6, value=due_date.strftime("%Y年%m月%d日")) 
cell.alignment = right_align
    
quotation_wb.save(f"../output/{target_id}_quotation.xlsx")